# Demo visualisation

In [47]:
import polygraphs as pg

from polygraphs import graphs
from polygraphs import hyperparameters as hparams
from polygraphs import visualisations as viz
from polygraphs import ops

%matplotlib inline

In [48]:
# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.01

params.network.kind = 'complete'
params.network.size = 8

# Enable logging; print progress every 100 steps
params.logging.enabled = True
params.logging.interval = 100

# Take snapshots (incl. messages)
params.snapshots.enabled = True
params.snapshots.interval = 1
params.snapshots.messages = True

params.simulation.steps = 0
params.simulation.repeats = 1

# Store results in directory
params.simulation.results = "data/demo-03"

# Set seed
params.seed = 0

pg.random(params.seed)
_ = pg.simulate(params, op=ops.BalaGoyalOp)

[MON] step 0001 Ksteps/s   0.00 A/B 0.62/0.38
[MON] step 0095 Ksteps/s   0.54 A/B 0.00/1.00
 INFO polygraphs> Sim #0001:     95 steps    0.18s; action: B undefined: 0 converged: 1 polarized: 0 


In [49]:
import os
import h5py
import json

import torch
import dgl
import networkx as nx


def __export_nodes(graph, directory):
    """
    Export nodes
    """
    filename = os.path.join(directory, "nodes.json")
    
    data = []
    for nid in graph.nodes():
        belief = graph.ndata["beliefs"][nid].item()
        data.append({"id": nid.item(), "belief": belief})
    
    with open(filename, "w") as fstream:
        json.dump(data, fstream, indent=4)
    return


def __export_links(graph, directory):
    """
    Export links
    """
    filename = os.path.join(directory, "links.json")

    data = []
    src, dst = graph.edges()
    for s, t in zip(src, dst):
        data.append({"source": s.item(), "target": t.item()})
    
    with open(filename, "w") as fstream:
        json.dump(data, fstream, indent=4)
    return


def __export_messages(graph, iid):
    """
    Export messages from a given iteration (`iid`).
    """
    data = []
    for u in graph.nodes():
        payoff = graph.ndata["payoffs"][u].item()
        sample = graph.ndata["samples"][u].item()
        
        # Ingore "empty" messages
        if sample < 1:
            continue

        # Iterate over node's neighbours 
        neighbours = graph.out_edges(u, form="uv")[1]
        for v in neighbours:
            data.append({"iid": iid, "source": u.item(), "target": v.item(), "payoff": payoff, "sample": sample})
    
    # Return messages from given iteration
    return data


def __export_beliefs(graph, iid):
    """
    Export beliefs from a given iteration (`iid`).
    """
    data = []
    for nid in graph.nodes():
        belief = graph.ndata["beliefs"][nid].item()
        data.append({"iid": iid, "id": nid.item(), "belief": belief})
    return data


def __export_iterations(graph, snapshots, directory):
    """
    Export iterations
    """
    # Output file
    filename = os.path.join(directory, "iterations.json")

    _keys = [int(key) for key in snapshots["beliefs"].keys()]
    _keys = sorted(_keys)

    data = []
    for key in _keys:
        # Populate graph node attributes
        graph.ndata["beliefs"] = torch.tensor(snapshots["beliefs"][str(key)][:])
        graph.ndata["payoffs"] = torch.tensor(snapshots["payoffs"][str(key)][:].T[0])
        graph.ndata["samples"] = torch.tensor(snapshots["payoffs"][str(key)][:].T[1])

        # Export messages
        data += __export_messages(graph, key)

        # Export new node beliefs
        data += __export_beliefs(graph, key)

    with open(filename, "w") as fstream:
        json.dump(data, fstream, indent=4)


def toJSON(directory, id):
    """
    Post-process graph snapshots and export them into JSON format
    """
    # Load DGL graph
    graphs, _ = dgl.load_graphs(os.path.join(directory, f"{id}.bin"))
    graph = graphs[0]

    # Remove self-loops
    graph = dgl.remove_self_loop(graph)

    # Export nodes
    __export_nodes(graph, directory)

    # Export links
    __export_links(graph, directory)

    # Read snapshots
    snapshots = h5py.File(os.path.join(directory, f"{id}.hd5"), "r")
    
    # Export iterations
    __export_iterations(graph, snapshots, directory)

    return


toJSON("data/demo-03", 1)